## **基于ResNeXt的树叶分类课程竞赛**

In [1]:
# !pip install torchcontrib 
# # 对优化器使用SWA

In [2]:
!pip install git+https://github.com/ildoonet/cutmix 
# 安装CutMix

  Cloning https://github.com/ildoonet/cutmix to c:\users\coin.lab\appdata\local\temp\pip-req-build-p3qiyl16
  Created wheel for cutmix: filename=cutmix-0.1-cp37-none-any.whl size=3642 sha256=f797baaa2877a815921027126cdec93672b35c5994cb9b6df9c8ab6ee00f89f0
  Stored in directory: C:\Users\coin.lab\AppData\Local\Temp\pip-ephem-wheel-cache-wiv6a_u1\wheels\8a\40\20\615302921d0fef73e55b17b5dd57169d4879dfe6dd7ad8ff50
Successfully built cutmix


  Running command git clone -q https://github.com/ildoonet/cutmix 'C:\Users\coin.lab\AppData\Local\Temp\pip-req-build-p3qiyl16'


In [3]:
!pip install ttach 
# Test-time Augmentation

In [4]:
# 导入各种包
import torch
import torch.nn as nn
from torch.nn import functional as F
import ttach as tta
from torchcontrib.optim import SWA

from cutmix.cutmix import CutMix
from cutmix.utils import CutMixCrossEntropyLoss

import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import transforms
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.model_selection import KFold
from PIL import Image
import os
import matplotlib.pyplot as plt
import torchvision.models as models
# This is for the progress bar.
from tqdm import tqdm


### **整理数据集**

### **数据读取与预处理**

In [5]:
# 看看label文件长啥样
labels_dataframe = pd.read_csv('../content/competitions/classify-leaves/train.csv')
labels_dataframe.head(5)

,image,label
0,images/0.jpg,maclura_pomifera
1,images/1.jpg,maclura_pomifera
2,images/2.jpg,maclura_pomifera
3,images/3.jpg,maclura_pomifera
4,images/4.jpg,maclura_pomifera


In [6]:
# 把label文件排个序
leaves_labels = sorted(list(set(labels_dataframe['label'])))
n_classes = len(leaves_labels)
print(n_classes)
leaves_labels[:10]

176


['abies_concolor',
 'abies_nordmanniana',
 'acer_campestre',
 'acer_ginnala',
 'acer_griseum',
 'acer_negundo',
 'acer_palmatum',
 'acer_pensylvanicum',
 'acer_platanoides',
 'acer_pseudoplatanus']

In [7]:
# 把label转成对应的数字
class_to_num = dict(zip(leaves_labels, range(n_classes)))
class_to_num

{'abies_concolor': 0,
 'abies_nordmanniana': 1,
 'acer_campestre': 2,
 'acer_ginnala': 3,
 'acer_griseum': 4,
 'acer_negundo': 5,
 'acer_palmatum': 6,
 'acer_pensylvanicum': 7,
 'acer_platanoides': 8,
 'acer_pseudoplatanus': 9,
 'acer_rubrum': 10,
 'acer_saccharinum': 11,
 'acer_saccharum': 12,
 'aesculus_flava': 13,
 'aesculus_glabra': 14,
 'aesculus_hippocastamon': 15,
 'aesculus_pavi': 16,
 'ailanthus_altissima': 17,
 'albizia_julibrissin': 18,
 'amelanchier_arborea': 19,
 'amelanchier_canadensis': 20,
 'amelanchier_laevis': 21,
 'asimina_triloba': 22,
 'betula_alleghaniensis': 23,
 'betula_jacqemontii': 24,
 'betula_lenta': 25,
 'betula_nigra': 26,
 'betula_populifolia': 27,
 'broussonettia_papyrifera': 28,
 'carpinus_betulus': 29,
 'carpinus_caroliniana': 30,
 'carya_cordiformis': 31,
 'carya_glabra': 32,
 'carya_ovata': 33,
 'carya_tomentosa': 34,
 'castanea_dentata': 35,
 'catalpa_bignonioides': 36,
 'catalpa_speciosa': 37,
 'cedrus_atlantica': 38,
 'cedrus_deodara': 39,
 'cedru

In [8]:
# 再转换回来，方便最后预测的时候使用
num_to_class = {v : k for k, v in class_to_num.items()}
num_to_class

{0: 'abies_concolor',
 1: 'abies_nordmanniana',
 2: 'acer_campestre',
 3: 'acer_ginnala',
 4: 'acer_griseum',
 5: 'acer_negundo',
 6: 'acer_palmatum',
 7: 'acer_pensylvanicum',
 8: 'acer_platanoides',
 9: 'acer_pseudoplatanus',
 10: 'acer_rubrum',
 11: 'acer_saccharinum',
 12: 'acer_saccharum',
 13: 'aesculus_flava',
 14: 'aesculus_glabra',
 15: 'aesculus_hippocastamon',
 16: 'aesculus_pavi',
 17: 'ailanthus_altissima',
 18: 'albizia_julibrissin',
 19: 'amelanchier_arborea',
 20: 'amelanchier_canadensis',
 21: 'amelanchier_laevis',
 22: 'asimina_triloba',
 23: 'betula_alleghaniensis',
 24: 'betula_jacqemontii',
 25: 'betula_lenta',
 26: 'betula_nigra',
 27: 'betula_populifolia',
 28: 'broussonettia_papyrifera',
 29: 'carpinus_betulus',
 30: 'carpinus_caroliniana',
 31: 'carya_cordiformis',
 32: 'carya_glabra',
 33: 'carya_ovata',
 34: 'carya_tomentosa',
 35: 'castanea_dentata',
 36: 'catalpa_bignonioides',
 37: 'catalpa_speciosa',
 38: 'cedrus_atlantica',
 39: 'cedrus_deodara',
 40: 'c

In [9]:
# 继承pytorch的dataset，创建自己的
class TrainValidData(Dataset):
    def __init__(self, csv_path, file_path, resize_height=224, resize_width=224, transform=None):
        """
        Args:
            csv_path (string): csv 文件路径
            img_path (string): 图像文件所在路径

        """
        
        # 需要调整后的照片尺寸，我这里每张图片的大小尺寸不一致#
        self.resize_height = resize_height
        self.resize_width = resize_width

        self.file_path = file_path
        self.to_tensor = transforms.ToTensor() #将数据转换成tensor形式
        self.transform = transform

        # 读取 csv 文件
        # 利用pandas读取csv文件
        self.data_info = pd.read_csv(csv_path, header=None)  #header=None是去掉表头部分
        # 文件第一列包含图像文件名称
        self.image_arr = np.asarray(self.data_info.iloc[1:,0]) #self.data_info.iloc[1:,0]表示读取第一列，从第二行开始一直读取到最后一行
        # 第二列是图像的 label
        self.label_arr = np.asarray(self.data_info.iloc[1:,1])
        # 计算 length
        self.data_len = len(self.data_info.index) - 1

    def __getitem__(self, index):
        # 从 image_arr中得到索引对应的文件名
        single_image_name = self.image_arr[index]

        # 读取图像文件
        img_as_img = Image.open(self.file_path + single_image_name)
        
        #如果需要将RGB三通道的图片转换成灰度图片可参考下面两行
        # if img_as_img.mode != 'L':
        #     img_as_img = img_as_img.convert('L')
        
        #设置好需要转换的变量，还可以包括一系列的nomarlize等等操作
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor()
        ])
        img_as_img = transform(img_as_img)

        # 得到图像的 label
        label = self.label_arr[index]
        number_label = class_to_num[label]

        return (img_as_img, number_label)  #返回每一个index对应的图片数据和对应的label

    def __len__(self):
        return self.data_len

In [10]:
# 继承pytorch的dataset，创建自己的
class TestData(Dataset):
    def __init__(self, csv_path, file_path, resize_height=224, resize_width=224, transform = None):
        """
        Args:
            csv_path (string): csv 文件路径
            img_path (string): 图像文件所在路径

        """
        
        # 需要调整后的照片尺寸，我这里每张图片的大小尺寸不一致#
        self.resize_height = resize_height
        self.resize_width = resize_width

        self.file_path = file_path
        self.transform = transform
        self.to_tensor = transforms.ToTensor() #将数据转换成tensor形式

        # 读取 csv 文件
        # 利用pandas读取csv文件
        self.data_info = pd.read_csv(csv_path, header=None)  #header=None是去掉表头部分
        # 文件第一列包含图像文件名称
        self.image_arr = np.asarray(self.data_info.iloc[1:,0]) #self.data_info.iloc[1:,0]表示读取第一列，从第二行开始一直读取到最后一行
        # 计算 length
        self.data_len = len(self.data_info.index) - 1
        
    def __getitem__(self, index):
        # 从 image_arr中得到索引对应的文件名
        single_image_name = self.image_arr[index]

        # 读取图像文件
        img_as_img = Image.open(self.file_path + single_image_name)
        
        #如果需要将RGB三通道的图片转换成灰度图片可参考下面两行
        # if img_as_img.mode != 'L':
        #     img_as_img = img_as_img.convert('L')
        
        #设置好需要转换的变量，还可以包括一系列的nomarlize等等操作
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor()
        ])
        img_as_img = transform(img_as_img)


        return img_as_img

    def __len__(self):
        return self.data_len

In [11]:
train_transform = transforms.Compose([
    # 随机裁剪图像，所得图像为原始面积的0.08到1之间，高宽比在3/4和4/3之间。
    # 然后，缩放图像以创建224 x 224的新图像
    transforms.RandomResizedCrop(224, scale=(0.08, 1.0), ratio=(3.0 / 4.0, 4.0 / 3.0)),
    transforms.RandomHorizontalFlip(),
    # 随机更改亮度，对比度和饱和度
    transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4),
    # 添加随机噪声
    transforms.ToTensor(),
    # 标准化图像的每个通道
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])
val_test_transform = transforms.Compose([
    transforms.Resize(256),
    # 从图像中心裁切224x224大小的图片
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])

In [12]:
train_val_path = '../content/competitions/classify-leaves/train.csv'
test_path = '../content/competitions/classify-leaves/test.csv'
# csv文件中已经images的路径了，因此这里只到上一级目录
img_path = '../content/competitions/classify-leaves/'

train_val_dataset = TrainValidData(train_val_path, img_path)
test_dataset = TestData(test_path, img_path, transform = val_test_transform)
print(train_val_dataset.data_info)
print(test_dataset.data_info)

                      0                        1
0                 image                    label
1          images/0.jpg         maclura_pomifera
2          images/1.jpg         maclura_pomifera
3          images/2.jpg         maclura_pomifera
4          images/3.jpg         maclura_pomifera
...                 ...                      ...
18349  images/18348.jpg          aesculus_glabra
18350  images/18349.jpg  liquidambar_styraciflua
18351  images/18350.jpg            cedrus_libani
18352  images/18351.jpg      prunus_pensylvanica
18353  images/18352.jpg          quercus_montana

[18354 rows x 2 columns]
                     0
0                image
1     images/18353.jpg
2     images/18354.jpg
3     images/18355.jpg
4     images/18356.jpg
...                ...
8796  images/27148.jpg
8797  images/27149.jpg
8798  images/27150.jpg
8799  images/27151.jpg
8800  images/27152.jpg

[8801 rows x 1 columns]


### **模型**

In [13]:
# 是否要冻住模型的前面一些层
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        model = model
        for param in model.parameters():
            param.requires_grad = False

# resnext50_32x4d模型
def resnext_model(num_classes, feature_extract = False, use_pretrained=True):

    model_ft = models.resnext50_32x4d(pretrained=use_pretrained)
    set_parameter_requires_grad(model_ft, feature_extract)
    num_ftrs = model_ft.fc.in_features
    model_ft.fc = nn.Sequential(nn.Linear(num_ftrs, num_classes))

    return model_ft

In [14]:
# 看一下是在cpu还是GPU上
def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'

device = get_device()
print(device)

cuda


In [15]:
!nvidia-smi

Mon Jun 21 14:51:52 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 456.71       Driver Version: 456.71       CUDA Version: 11.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN RTX          WDDM  | 00000000:65:00.0 Off |                  N/A |
| 41%   50C    P8    28W / 280W |   1400MiB / 24576MiB |      4%      Default |
+-------------------------------+----------------------+----------------------+
|   1  TITAN RTX          WDDM  | 00000000:B3:00.0 Off |                  N/A |
| 40%   40C    P8    28W / 280W |   1400MiB / 24576MiB |      2%      Default |
+-------------------------------+----------------------+----------------------+
        

In [16]:
# Configuration options
k_folds = 5
num_epochs = 30
learning_rate = 1e-3
weight_decay = 1e-3
train_loss_function = CutMixCrossEntropyLoss(True)
valid_loss_function = nn.CrossEntropyLoss()
# For fold results
results = {}

# Set fixed random number seed
torch.manual_seed(42)

# Define the K-fold Cross Validator
kfold = KFold(n_splits=k_folds, shuffle=True)

### **训练**

In [17]:
# Start print
print('--------------------------------------')

# K-fold Cross Validation model evaluation
for fold, (train_ids,valid_ids) in enumerate(kfold.split(train_val_dataset)):

  # Print
  print(f'FOLD {fold}')
  print('--------------------------------------')

  # Sample elements randomly from a given list of ids, no replacement.
  train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
  valid_subsampler = torch.utils.data.SubsetRandomSampler(valid_ids)

  # Define data loaders for training and testing data in this fold
  trainloader = torch.utils.data.DataLoader(
                      CutMix(TrainValidData(train_val_path, img_path, transform = train_transform), num_class=176, beta=1.0, prob=0.5, num_mix=2), 
                      batch_size=128, sampler=train_subsampler, num_workers=0)
  validloader = torch.utils.data.DataLoader(
                      TrainValidData(train_val_path, img_path, transform = val_test_transform),
                      batch_size=128, sampler=valid_subsampler, num_workers=0)
  
  # Initialize a model and put it on the device specified.
  model = resnext_model(176)
  model = model.to(device)
  model.device = device
  
  # Initialize optimizer
  optimizer = torch.optim.AdamW(model.parameters(),lr=learning_rate,weight_decay= weight_decay)
#   optimizer = SWA(our_optimizer, swa_start=5, swa_freq =5, swa_lr=0.05)
  scheduler = CosineAnnealingLR(optimizer,T_max=10)

  # Run the training loop for defined number of epochs
  for epoch in range(0,num_epochs):
    model.train()
    # Print epoch
    print(f'Starting epoch {epoch+1}')
    # These are used to record information in training
    train_losses = []
    train_accs = []
    # Iterate the training set by batches
    for batch in tqdm(trainloader):
      # Move images and labels to GPU
      imgs, labels = batch
      imgs = imgs.to(device)
      labels = labels.to(device)
      # Forward the data
      logits = model(imgs)
      # Calculate loss
      loss = train_loss_function(logits,labels)
      # Clear gradients in previous step
      optimizer.zero_grad()
      # Compute gradients for parameters
      loss.backward()
      # Update the parameters with computed gradients
      optimizer.step()
      # Compute the accuracy for current batch.
      # acc = (logits.argmax(dim=-1) == labels).float().mean()
      # Record the loss and accuracy.
      train_losses.append(loss.item())
      # train_accs.append(acc)
    print("第%d个epoch的学习率：%f" % (epoch+1,optimizer.param_groups[0]['lr']))
    scheduler.step()
    # The average loss and accuracy of the training set is the average of the recorded values.
    train_loss = np.sum(train_losses) / len(train_losses)
    # train_acc = np.sum(train_accs) / len(train_accs)
    # Print the information.
    # print(f"[ Train | {epoch + 1:03d}/{num_epochs:03d} ] loss = {train_loss:.5f}, acc = {train_acc:.5f}")
    print(f"[ Train | {epoch + 1:03d}/{num_epochs:03d} ] loss = {train_loss:.5f}")

  # Train process (all epochs) is complete
  print('Training process has finished. Saving trained model.')
  print('Starting validation')

  # Saving the model
  print('saving model with loss {:.3f}'.format(train_loss))
  save_path = f'./model-fold-{fold}.pth'
  torch.save(model.state_dict(),save_path)
  # Start Validation
  model.eval()
  valid_losses = []
  valid_accs = []
  with torch.no_grad():
    for batch in tqdm(validloader):
      imgs, labels = batch
      # No gradient in validation
      logits = model(imgs.to(device))
      loss = valid_loss_function(logits,labels.to(device))
      acc = (logits.argmax(dim=-1) == labels.to(device)).float().mean()
      # Record loss and accuracy
      valid_losses.append(loss.item())        
      valid_accs.append(acc)
    # The average loss and accuracy
    valid_loss = np.sum(valid_losses)/len(valid_losses)
    valid_acc = np.sum(valid_accs)/len(valid_accs)
    print(f"[ Valid | {epoch + 1:03d}/{num_epochs:03d} ] loss = {valid_loss:.5f}, acc = {valid_acc:.5f}")
    print('Accuracy for fold %d: %d' % (fold, valid_acc))
    print('--------------------------------------')
    results[fold] = valid_acc
# Print fold results
print(f'K-FOLD CROSS VALIDATION RESULTS FOR {k_folds} FOLDS')
print('--------------------------------')
total_summation = 0.0
for key, value in results.items():
  print(f'Fold {key}: {value} ')
  total_summation += value
print(f'Average: {total_summation/len(results.items())} ')

--------------------------------------
FOLD 0
--------------------------------------
Starting epoch 1


  0%|          | 0/115 [00:00<?, ?it/s]C:\Users\coin.lab\Anaconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
100%|██████████| 115/115 [01:50<00:00,  1.04it/s]


第1个epoch的学习率：0.001000
[ Train | 001/030 ] loss = 3.53967
Starting epoch 2


100%|██████████| 115/115 [01:49<00:00,  1.05it/s]


第2个epoch的学习率：0.000976
[ Train | 002/030 ] loss = 2.53754
Starting epoch 3


...

第29个epoch的学习率：0.000095
[ Train | 029/030 ] loss = 1.01589
Starting epoch 30


100%|██████████| 115/115 [01:50<00:00,  1.04it/s]


第30个epoch的学习率：0.000024
[ Train | 030/030 ] loss = 0.99554
Training process has finished. Saving trained model.
Starting validation
saving model with loss 0.996


100%|██████████| 29/29 [00:10<00:00,  2.80it/s]


[ Valid | 030/030 ] loss = 0.04532, acc = 0.98761
Accuracy for fold 4: 0
--------------------------------------
K-FOLD CROSS VALIDATION RESULTS FOR 5 FOLDS
--------------------------------
Fold 0: 0.9771167635917664 
Fold 1: 0.9862607717514038 
Fold 2: 0.9851831793785095 
Fold 3: 0.985722005367279 
Fold 4: 0.9876077771186829 
Average: 0.9843781590461731 


### **预测**

In [18]:
testloader = torch.utils.data.DataLoader(
                      TestData(test_path, img_path, transform = val_test_transform),
                      batch_size=128, num_workers=0)

In [19]:
## predict
model = resnext_model(176)

# create model and load weights from checkpoint
model = model.to(device)
# load the all folds
for test_fold in range(k_folds):
  model_path = f'./model-fold-{test_fold}.pth'
  saveFileName = f'./submission-fold-{test_fold}.csv'
  model.load_state_dict(torch.load(model_path))

  # Make sure the model is in eval mode.
  # Some modules like Dropout or BatchNorm affect if the model is in training mode.
  model.eval()
  tta_model = tta.ClassificationTTAWrapper(model, tta.aliases.five_crop_transform(200,200))

  # Initialize a list to store the predictions.
  predictions = []
  # Iterate the testing set by batches.
  for batch in tqdm(testloader):
      
      imgs = batch
      with torch.no_grad():
          logits = tta_model(imgs.to(device))
      
      # Take the class with greatest logit as prediction and record it.
      predictions.extend(logits.argmax(dim=-1).cpu().numpy().tolist())

  preds = []
  for i in predictions:
      preds.append(num_to_class[i])

  test_data = pd.read_csv(test_path)
  test_data['label'] = pd.Series(preds)
  submission = pd.concat([test_data['image'], test_data['label']], axis=1)
  submission.to_csv(saveFileName, index=False)
  print("ResNeXt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!")

100%|██████████| 69/69 [01:16<00:00,  1.11s/it]


ResNeXt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


100%|██████████| 69/69 [01:01<00:00,  1.12it/s]


ResNeXt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


100%|██████████| 69/69 [01:02<00:00,  1.11it/s]


ResNeXt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


100%|██████████| 69/69 [01:02<00:00,  1.11it/s]


ResNeXt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


100%|██████████| 69/69 [01:01<00:00,  1.12it/s]


ResNeXt Model Results Done!!!!!!!!!!!!!!!!!!!!!!!!!!!


### **最终投票Bagging**

In [20]:
df0 = pd.read_csv('./submission-fold-0.csv')
df1 = pd.read_csv('./submission-fold-1.csv')
df2 = pd.read_csv('./submission-fold-2.csv')
df3 = pd.read_csv('./submission-fold-3.csv')
df4 = pd.read_csv('./submission-fold-4.csv')

In [21]:
list_num_label0 = []
for i in df0['label']:
  list_num_label0.append(class_to_num[i])
df0['num_label0']=list_num_label0
df0.head()

,image,label,num_label0
0,images/18353.jpg,asimina_triloba,22
1,images/18354.jpg,betula_nigra,26
2,images/18355.jpg,platanus_acerifolia,120
3,images/18356.jpg,pinus_bungeana,102
4,images/18357.jpg,platanus_acerifolia,120


In [22]:
list_num_label1 = []
for i in df1['label']:
  list_num_label1.append(class_to_num[i])
df1['num_label1']=list_num_label1
df1.head()

,image,label,num_label1
0,images/18353.jpg,asimina_triloba,22
1,images/18354.jpg,betula_nigra,26
2,images/18355.jpg,platanus_acerifolia,120
3,images/18356.jpg,pinus_bungeana,102
4,images/18357.jpg,platanus_acerifolia,120


In [23]:
list_num_label2 = []
for i in df2['label']:
  list_num_label2.append(class_to_num[i])
df2['num_label2']=list_num_label2
df2.head()

,image,label,num_label2
0,images/18353.jpg,asimina_triloba,22
1,images/18354.jpg,platanus_occidentalis,121
2,images/18355.jpg,platanus_acerifolia,120
3,images/18356.jpg,pinus_bungeana,102
4,images/18357.jpg,platanus_acerifolia,120


In [24]:
list_num_label3 = []
for i in df3['label']:
  list_num_label3.append(class_to_num[i])
df3['num_label3']=list_num_label3
df3.head()

,image,label,num_label3
0,images/18353.jpg,asimina_triloba,22
1,images/18354.jpg,betula_nigra,26
2,images/18355.jpg,platanus_acerifolia,120
3,images/18356.jpg,pinus_bungeana,102
4,images/18357.jpg,platanus_acerifolia,120


In [25]:
list_num_label4 = []
for i in df4['label']:
  list_num_label4.append(class_to_num[i])
df4['num_label4']=list_num_label4
df4.head()

,image,label,num_label4
0,images/18353.jpg,asimina_triloba,22
1,images/18354.jpg,betula_nigra,26
2,images/18355.jpg,platanus_acerifolia,120
3,images/18356.jpg,pinus_bungeana,102
4,images/18357.jpg,platanus_acerifolia,120


In [26]:
df_all = df0.copy()
df_all.drop(['label'],axis=1,inplace=True)
df_all.head()

,image,num_label0
0,images/18353.jpg,22
1,images/18354.jpg,26
2,images/18355.jpg,120
3,images/18356.jpg,102
4,images/18357.jpg,120


In [27]:
df_all['num_label1']=list_num_label1
df_all['num_label2']=list_num_label2
df_all['num_label3']=list_num_label3
df_all['num_label4']=list_num_label4
df_all.head()

,image,num_label0,num_label1,num_label2,num_label3,num_label4
0,images/18353.jpg,22,22,22,22,22
1,images/18354.jpg,26,26,121,26,26
2,images/18355.jpg,120,120,120,120,120
3,images/18356.jpg,102,102,102,102,102
4,images/18357.jpg,120,120,120,120,120


In [28]:
df_all_transpose = df_all.copy().drop(['image'],axis=1).transpose()
df_all_transpose.head()

,0,1,2,3,4,5,6,7,8,9,...,8790,8791,8792,8793,8794,8795,8796,8797,8798,8799
num_label0,22,26,120,102,120,74,84,91,165,169,...,117,53,117,53,73,117,53,117,73,73
num_label1,22,26,120,102,120,74,83,91,165,169,...,117,53,117,53,73,119,53,117,73,73
num_label2,22,121,120,102,120,74,84,91,165,169,...,117,53,117,53,73,119,53,117,73,73
num_label3,22,26,120,102,120,74,83,91,165,169,...,117,53,117,53,73,119,53,117,73,73
num_label4,22,26,120,102,120,74,84,91,165,169,...,117,53,117,53,73,117,53,117,73,73


In [29]:
df_mode = df_all_transpose.mode().transpose()
df_mode.head()

,0,1,2,3,4
0,22.0,NaN,NaN,NaN,NaN
1,26.0,NaN,NaN,NaN,NaN
2,120.0,NaN,NaN,NaN,NaN
3,102.0,NaN,NaN,NaN,NaN
4,120.0,NaN,NaN,NaN,NaN


In [30]:
voting_class = []
for each in df_mode[0]:
  voting_class.append(num_to_class[each])
voting_class

['asimina_triloba',
 'betula_nigra',
 'platanus_acerifolia',
 'pinus_bungeana',
 'platanus_acerifolia',
 'koelreuteria_paniculata',
 'magnolia_tripetala',
 'metasequoia_glyptostroboides',
 'tilia_cordata',
 'ulmus_americana',
 'quercus_marilandica',
 'quercus_muehlenbergii',
 'asimina_triloba',
 'castanea_dentata',
 'pinus_echinata',
 'metasequoia_glyptostroboides',
 'picea_pungens',
 'quercus_michauxii',
 'pinus_cembra',
 'cryptomeria_japonica',
 'fraxinus_pennsylvanica',
 'morus_alba',
 'cercis_canadensis',
 'ptelea_trifoliata',
 'halesia_tetraptera',
 'liriodendron_tulipifera',
 'acer_rubrum',
 'pinus_bungeana',
 'quercus_montana',
 'cornus_mas',
 'magnolia_tripetala',
 'ficus_carica',
 'carya_glabra',
 'quercus_palustris',
 'prunus_virginiana',
 'prunus_serotina',
 'magnolia_macrophylla',
 'acer_palmatum',
 'picea_orientalis',
 'abies_concolor',
 'quercus_virginiana',
 'liriodendron_tulipifera',
 'stewartia_pseudocamellia',
 'pinus_pungens',
 'gleditsia_triacanthos',
 'stewartia_ps

In [31]:
df_all['label'] = voting_class
df_all.head()

,image,num_label0,num_label1,num_label2,num_label3,num_label4,label
0,images/18353.jpg,22,22,22,22,22,asimina_triloba
1,images/18354.jpg,26,26,121,26,26,betula_nigra
2,images/18355.jpg,120,120,120,120,120,platanus_acerifolia
3,images/18356.jpg,102,102,102,102,102,pinus_bungeana
4,images/18357.jpg,120,120,120,120,120,platanus_acerifolia


In [32]:
df_submission = df_all[['image','label']].copy()
df_submission.head()

,image,label
0,images/18353.jpg,asimina_triloba
1,images/18354.jpg,betula_nigra
2,images/18355.jpg,platanus_acerifolia
3,images/18356.jpg,pinus_bungeana
4,images/18357.jpg,platanus_acerifolia


In [33]:
df_submission.to_csv('./submission-voting.csv', index=False)
print('Final voting results successfully saved!')

Final voting results successfully saved!
